In [2]:
!pip install dgl
from os import listdir
import dgl
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

     |████████████████████████████████| 3.0MB 3.5MB/s 


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
filename = "/content/drive/My Drive/adaptec1.nets"

edge_file = "/content/add_edges.py"
edge_file_handle = open(edge_file, "w+")

node_count = 0

with open(filename,encoding="utf8") as f:
    netSection = False
    edges = 0
    lineCount = 0
    src=[]
    dest=[]
    for line in f:
        lineCount += 1
        #Newline
        if len(line) < 2:
            continue
            
        # Comments
        if line[0] == '#':
            continue
        #print(line)
        splitLine = line.split()
        #print("Words = ", len(splitLine))
        if(splitLine[0] == 'NetDegree'):
            
            if edges > 0: #Previous NetDegree not complete!
                print("Missed an edge at line: ", lineCount-1)
                
            edges = int(splitLine[2])
            netSection = True
            src=[]
            dest=[]
            #G.add_nodes(num)
            
        elif netSection == True:
          node_name = splitLine[0][1:]
          node_val  = int(node_name)
          if node_count < node_val+1:
              node_count = node_val+1
          
          if(splitLine[1] == 'O'):
              src.append(node_name)
              edges -= 1
          elif(splitLine[1] == 'I'):
              dest.append(node_name)
              edges -= 1
      
        if netSection == True:
            if edges == 0:
                if (len(src)<1):
                    print("No source edge, skipping")
                    continue
                if (len(dest)<1):
                    print("No dest edge, skipping")
                    continue
                    
                netSection = False
                for des_idx in range(len(dest)):
                    cmd = "G.add_edges("+src[0]+","+dest[des_idx]+")"+"\n"
                    edge_file_handle.write(cmd)
                    
        #if lineCount > 100:
        #    break


edge_file_handle.close()


No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584552
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584660
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584723
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584753
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584781
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584978
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585078
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585138
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585199
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585306
No dest edge, skipping
No dest edge, skipping
Missed an edge at line: 

In [5]:
G = dgl.DGLGraph()
G.add_nodes(node_count)
exec(open(edge_file).read())
print("Nodes = ", G.number_of_nodes())
print("Edges = ", G.number_of_edges())

Nodes =  211447
Edges =  720134


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
embed = nn.Embedding(G.number_of_nodes(),6)

In [0]:
G.ndata['feat'] = embed.weight

In [0]:
from dgl.nn.pytorch import GraphConv

In [0]:
class GCN(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(GCN,self).__init__()
        self.conv1 = GraphConv(input_size,hidden_size)
        self.conv2 = GraphConv(hidden_size,num_classes)
    
    def forward(self,G,inputs):
        h = self.conv1(G,inputs)
        h = torch.relu(h) 
        h = self.conv2(G,h)
        return h

In [0]:
net = GCN(6,5,20)

In [0]:
inputs = embed.weight
a = np.linspace(0,G.number_of_nodes()-1,20,dtype=int)
labelled_nodes = torch.tensor(a)
labels = torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

# Training Model

In [13]:
import itertools
import time

optimizer = torch.optim.Adam(itertools.chain(net.parameters(), embed.parameters()), lr=0.01)
all_logits = []

start_time = time.time()

for epoch in range(210):
    logits = net(G, inputs)
    all_logits.append(logits.detach())
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[labelled_nodes], labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print('Epoch {} | Loss: {}'.format(epoch + 1, loss.item()))

end_time = time.time()
print("Training Time: {}".format(end_time - start_time))

Epoch 1 | Loss: 3.041616678237915
Epoch 2 | Loss: 3.0164990425109863
Epoch 3 | Loss: 2.992506742477417
Epoch 4 | Loss: 2.9694736003875732
Epoch 5 | Loss: 2.9474291801452637
Epoch 6 | Loss: 2.925947666168213
Epoch 7 | Loss: 2.904907703399658
Epoch 8 | Loss: 2.884082078933716
Epoch 9 | Loss: 2.863351345062256
Epoch 10 | Loss: 2.8432252407073975
Epoch 11 | Loss: 2.8227696418762207
Epoch 12 | Loss: 2.802098274230957
Epoch 13 | Loss: 2.78123140335083
Epoch 14 | Loss: 2.7600347995758057
Epoch 15 | Loss: 2.7382349967956543
Epoch 16 | Loss: 2.715729236602783
Epoch 17 | Loss: 2.692836046218872
Epoch 18 | Loss: 2.669405460357666
Epoch 19 | Loss: 2.6454293727874756
Epoch 20 | Loss: 2.620826482772827
Epoch 21 | Loss: 2.5955653190612793
Epoch 22 | Loss: 2.5695641040802
Epoch 23 | Loss: 2.5429399013519287
Epoch 24 | Loss: 2.5157904624938965
Epoch 25 | Loss: 2.4879822731018066
Epoch 26 | Loss: 2.459378242492676
Epoch 27 | Loss: 2.4300293922424316
Epoch 28 | Loss: 2.3998923301696777
Epoch 29 | Loss: 2

In [0]:
pos = {}
cluster_nodes = {} # Every value in key->value pair includes the cluster centre node as well.
for v in range(G.number_of_nodes()):
  pos[v] = all_logits[99][v].numpy()
  cls = pos[v].argmax()
  if cluster_nodes.get(cls) :
    cluster_nodes[cls].append(v)
  else:
    cluster_nodes[cls] = [] 
    cluster_nodes[cls].append(v)

In [15]:
print(cluster_nodes)

{0: [0, 678, 1652, 1684, 2564, 3463, 3864, 3867, 4279, 4368, 4549, 4638, 4653, 4814, 4830, 5083, 5093, 5140, 5299, 5320, 5324, 5335, 5593, 5625, 5638, 5653, 5663, 5700, 5703, 5705, 5730, 5740, 5769, 5788, 5836, 5909, 6008, 6170, 6337, 6875, 7089, 7122, 8388, 8468, 9029, 9064, 9079, 9216, 9279, 9805, 9808, 10386, 10520, 10989, 11060, 11062, 11077, 11117, 11201, 11204, 11231, 11235, 11236, 11277, 11386, 11412, 11428, 11431, 11459, 11535, 11666, 11687, 11897, 11996, 12323, 12324, 12418, 12676, 12745, 12857, 12911, 13031, 13151, 13167, 13911, 13972, 14482, 14595, 14876, 14967, 15111, 15128, 15184, 15246, 15337, 16106, 16285, 16357, 16677, 16777, 17108, 17116, 17130, 17239, 17255, 17259, 17494, 17593, 17611, 17830, 17831, 17832, 17839, 18111, 18244, 19025, 19058, 19557, 19564, 19565, 19877, 20100, 20217, 20687, 20831, 20927, 21343, 22139, 22211, 22243, 22351, 22703, 22932, 22938, 23333, 23470, 23570, 23598, 23749, 23777, 23795, 23844, 23848, 24122, 24367, 24434, 24850, 24902, 25315, 26161, 

# Cluster Quality Metric #2 : Number of Nodes per Cluster Distribution

In [0]:
def number_of_nodes_per_cluster():
  number_dict = {} # Dictionary mapping the node label to the number of nodes in a particular cluster
  for key,value in cluster_nodes.items():
    number_dict[key] = len(value)

  number_dict = sorted(number_dict.items()) # Sorting dictionary by keys in increasing order

  for ix in number_dict:
    print("Number of Nodes in Cluster {} = {}\n".format(ix[0],ix[1]))

  return number_dict

In [17]:
number_dict = number_of_nodes_per_cluster()

Number of Nodes in Cluster 0 = 1496

Number of Nodes in Cluster 1 = 5625

Number of Nodes in Cluster 2 = 3808

Number of Nodes in Cluster 3 = 5239

Number of Nodes in Cluster 4 = 2606

Number of Nodes in Cluster 5 = 13638

Number of Nodes in Cluster 6 = 1078

Number of Nodes in Cluster 7 = 6117

Number of Nodes in Cluster 8 = 4100

Number of Nodes in Cluster 9 = 19248

Number of Nodes in Cluster 10 = 47795

Number of Nodes in Cluster 11 = 16505

Number of Nodes in Cluster 12 = 41830

Number of Nodes in Cluster 13 = 463

Number of Nodes in Cluster 14 = 6666

Number of Nodes in Cluster 15 = 9153

Number of Nodes in Cluster 16 = 3985

Number of Nodes in Cluster 17 = 13338

Number of Nodes in Cluster 18 = 7353

Number of Nodes in Cluster 19 = 1404



# HashMap (Dictionary) for easy visualization of networkx graph
- Dictionary to map node numbers in a particular cluster to a standard count procedure of 0 to (n-1) 
- n = number of nodes in a particular cluster
- Cluster label is taken as input from user

In [0]:
num = int(input("Enter cluster number: "))

Enter cluster number: 10


In [0]:
standard_nodes_dict = {}

count = 0

for node in cluster_nodes[num]:
  standard_nodes_dict[node] = count
  count += 1

In [0]:
print(standard_nodes_dict.items())

dict_items([(44, 0), (45, 1), (48, 2), (49, 3), (88, 4), (89, 5), (90, 6), (137, 7), (140, 8), (192, 9), (193, 10), (194, 11), (200, 12), (201, 13), (202, 14), (232, 15), (233, 16), (234, 17), (256, 18), (257, 19), (258, 20), (263, 21), (280, 22), (281, 23), (282, 24), (284, 25), (285, 26), (286, 27), (296, 28), (297, 29), (298, 30), (303, 31), (345, 32), (346, 33), (347, 34), (357, 35), (358, 36), (359, 37), (387, 38), (388, 39), (389, 40), (396, 41), (397, 42), (398, 43), (443, 44), (459, 45), (462, 46), (503, 47), (504, 48), (505, 49), (539, 50), (540, 51), (541, 52), (557, 53), (558, 54), (559, 55), (563, 56), (564, 57), (565, 58), (623, 59), (629, 60), (638, 61), (682, 62), (683, 63), (704, 64), (713, 65), (725, 66), (739, 67), (802, 68), (805, 69), (806, 70), (849, 71), (931, 72), (933, 73), (1060, 74), (1061, 75), (1068, 76), (1069, 77), (1113, 78), (1132, 79), (1135, 80), (1152, 81), (1156, 82), (1236, 83), (1242, 84), (1295, 85), (1296, 86), (1301, 87), (1302, 88), (1321, 89),

# Clustering Visualisation: Cluster Quality Metric # 3
- Visualisation for any generic cluster label taken as input from user

In [0]:
# dict_keys = list(standard_nodes_dict.keys())

# input_edges = G.in_edges(dict_keys, form='uv')

# ax = input_edges[0].numpy()

# ax = [i for i in ax if i in standard_nodes_dict.keys()]

# print(ax)

In [0]:
src = []
dst = []

In [0]:
def build_graph():

  dict_keys = list(standard_nodes_dict.keys())

  input_edges = G.in_edges(dict_keys, form='uv')

  src = input_edges[0].numpy()
  dst = input_edges[1].numpy()

  m = src.shape[0]
  src1 = []
  dst1 = []

  for ix in range(m):
    if src[ix] not in standard_nodes_dict.keys():
      continue
    else:
      src1.append(src[ix])
      dst1.append(dst[ix])

  src1 = np.array(src1)
  dst1 = np.array(dst1)

  j = 0
  for num in src1:
    num = standard_nodes_dict.get(num)
    src1[j] = num
    j += 1

  i = 0
  for num1 in dst1:
      num1 = standard_nodes_dict[num1]
      dst1[i] = num1
      i += 1

  u = np.concatenate([src1,dst1])
  v = np.concatenate([dst1,src1])

  return dgl.DGLGraph((u,v))

In [0]:
G1 = build_graph()

In [0]:
print("Nodes = {}".format(G1.number_of_nodes()))
print("Edges = {}".format(G1.number_of_edges()))

Nodes = 14718
Edges = 14738


In [0]:
nx_G1 = G1.to_networkx().to_undirected()

In [0]:
# nx.write_edgelist(nx_G1,"/content/cluster14.csv")

In [0]:
# pos1 = nx.kamada_kawai_layout(nx_G1) # Session crashes on running this cell

In [0]:
# plt.style.use("seaborn")

In [0]:
# nx.draw(nx_G1,pos1,with_labels=True,node_color=[[0.7,0.7,0.7]])
# plt.show()

# Cluster Quality Metric # 1: Total number of edges between nodes which belong to different clusters divided by total edges represented as percentage (%)

In [18]:
m = 10 # Total number of clusters

edges_between_clusters = {}

for i in range(m):
  print("Calculating for cluster {}\n".format(i))
  nodes_i = cluster_nodes[i]

  for j in range(i+1,m):
    nodes_j = np.array(cluster_nodes[j])

    ij_input_edges = G.in_edges(nodes_i,form='uv')
    edges_src = np.array(ij_input_edges[0])
    
    count = 0

    for val1 in nodes_j:
      for val2 in edges_src:
        if val1 == val2:
          count += 1

    # for x in range(np.array(nodes_i).shape[0]):
    #   if edges_src[x] == nodes_j[x]:
    #     count += 1

    edges_between_clusters[(i,j)] = count

Calculating for cluster 0

Calculating for cluster 1

Calculating for cluster 2

Calculating for cluster 3

Calculating for cluster 4

Calculating for cluster 5

Calculating for cluster 6

Calculating for cluster 7

Calculating for cluster 8

Calculating for cluster 9



In [19]:
edges_between_clusters

{(0, 1): 175,
 (0, 2): 144,
 (0, 3): 143,
 (0, 4): 62,
 (0, 5): 376,
 (0, 6): 19,
 (0, 7): 206,
 (0, 8): 161,
 (0, 9): 553,
 (1, 2): 208,
 (1, 3): 435,
 (1, 4): 173,
 (1, 5): 867,
 (1, 6): 71,
 (1, 7): 441,
 (1, 8): 365,
 (1, 9): 1305,
 (2, 3): 349,
 (2, 4): 189,
 (2, 5): 753,
 (2, 6): 60,
 (2, 7): 374,
 (2, 8): 296,
 (2, 9): 956,
 (3, 4): 91,
 (3, 5): 715,
 (3, 6): 60,
 (3, 7): 210,
 (3, 8): 245,
 (3, 9): 891,
 (4, 5): 331,
 (4, 6): 11,
 (4, 7): 235,
 (4, 8): 155,
 (4, 9): 482,
 (5, 6): 328,
 (5, 7): 999,
 (5, 8): 1045,
 (5, 9): 3472,
 (6, 7): 15,
 (6, 8): 33,
 (6, 9): 76,
 (7, 8): 541,
 (7, 9): 1623,
 (8, 9): 629}

In [0]:
percentage_dict = {}

for key,value in edges_between_clusters.items():
  value = (value/G.number_of_edges())*100
  value = round(value,4)
  percentage_dict[key] = value

In [21]:
percentage_dict

{(0, 1): 0.0243,
 (0, 2): 0.02,
 (0, 3): 0.0199,
 (0, 4): 0.0086,
 (0, 5): 0.0522,
 (0, 6): 0.0026,
 (0, 7): 0.0286,
 (0, 8): 0.0224,
 (0, 9): 0.0768,
 (1, 2): 0.0289,
 (1, 3): 0.0604,
 (1, 4): 0.024,
 (1, 5): 0.1204,
 (1, 6): 0.0099,
 (1, 7): 0.0612,
 (1, 8): 0.0507,
 (1, 9): 0.1812,
 (2, 3): 0.0485,
 (2, 4): 0.0262,
 (2, 5): 0.1046,
 (2, 6): 0.0083,
 (2, 7): 0.0519,
 (2, 8): 0.0411,
 (2, 9): 0.1328,
 (3, 4): 0.0126,
 (3, 5): 0.0993,
 (3, 6): 0.0083,
 (3, 7): 0.0292,
 (3, 8): 0.034,
 (3, 9): 0.1237,
 (4, 5): 0.046,
 (4, 6): 0.0015,
 (4, 7): 0.0326,
 (4, 8): 0.0215,
 (4, 9): 0.0669,
 (5, 6): 0.0455,
 (5, 7): 0.1387,
 (5, 8): 0.1451,
 (5, 9): 0.4821,
 (6, 7): 0.0021,
 (6, 8): 0.0046,
 (6, 9): 0.0106,
 (7, 8): 0.0751,
 (7, 9): 0.2254,
 (8, 9): 0.0873}